In [1]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv

load_dotenv()

AIRTABLE_API_KEY = os.getenv("pata6B6aXqsEi1Z2z.bb94655c01c369fc867f60c8b97e812ea6514fec77a52eb5975118fc5a864e8e")
AIRTABLE_BASE_ID = os.getenv("AIRTABLE_BASE_ID")
AIRTABLE_TABLE_NAME = os.getenv("AIRTABLE_TABLE_NAME")

In [2]:
file_path = "inventory_data.xlsx"
xls = pd.ExcelFile(file_path)
print(xls.sheet_names)

['Chronology', 'Legend', 'Current Inventory ', 'Combos skus', 'Msku With Skus', 'Outbound', 'Returns', 'Outflow', 'Inwards', 'inventory Upload FK And Meesho', 'Inventory Upload Amazon', 'packaging msku', 'Location ']


In [3]:
# Load the master Excel file with all warehouse and marketplace data
file_path = "inventory_data.xlsx"

# Load multiple sheets from the file
sheets_dict = pd.read_excel(file_path, sheet_name=None)

# Display available sheet names
print("Sheets available:", sheets_dict.keys())

# Load specific sheets
df_inventory = sheets_dict['Current Inventory ']  # Current inventory across all warehouses
df_combos = sheets_dict['Combos skus']  # Combo SKU breakdown
df_msku_mapping = sheets_dict['Msku With Skus']  # Mapping of SKUs to Master SKUs
df_outbound = sheets_dict['Outbound']  # Outbound orders
df_returns = sheets_dict['Returns']  # Return inventory
df_outflow = sheets_dict['Outflow']  # Outflows from stock
df_inwards = sheets_dict['Inwards']  # Incoming stock
df_packaging = sheets_dict['packaging msku']  # Additional items like battery, covers
df_location = sheets_dict['Location ']  # Inventory location-wise distribution

# Load 7 days sales data from different marketplaces
df_sales_amazon = pd.read_csv("cste amazon.csv")
df_sales_flipkart = pd.read_csv("Cste FK.csv")
df_sales_meesho = pd.read_csv("cste meesho.csv")

Sheets available: dict_keys(['Chronology', 'Legend', 'Current Inventory ', 'Combos skus', 'Msku With Skus', 'Outbound', 'Returns', 'Outflow', 'Inwards', 'inventory Upload FK And Meesho', 'Inventory Upload Amazon', 'packaging msku', 'Location '])


In [4]:
df_inventory = sheets_dict['Current Inventory '].iloc[1:]  # Skip the first row
df_inventory.columns = sheets_dict['Current Inventory '].iloc[0]  # Set the correct header
df_inventory.reset_index(drop=True, inplace=True)

In [5]:
sku_mapping = dict(zip(df_msku_mapping['sku'], df_msku_mapping['msku']))

df_sales_amazon["Master SKU"] = df_sales_amazon["MSKU"].map(sku_mapping)
df_sales_flipkart["Master SKU"] = df_sales_flipkart["SKU"].map(sku_mapping)
df_sales_meesho["Master SKU"] = df_sales_meesho["SKU"].map(sku_mapping)

In [9]:
print("Columns in df_sales_amazon:", df_sales_flipkart.columns.tolist())

Columns in df_sales_amazon: ['Ordered On', 'Shipment ID', 'ORDER ITEM ID', 'Order Id', 'HSN CODE', 'Order State', 'Order Type', 'FSN', 'SKU', 'Product', 'Invoice No.', 'CGST', 'IGST', 'SGST', 'Invoice Date (mm/dd/yy)', 'Invoice Amount', 'Selling Price Per Item', 'Shipping and Handling Charges', 'Quantity', 'Price inc. FKMP Contribution & Subsidy', 'Buyer name', 'Ship to name', 'Address Line 1', 'Address Line 2', 'City', 'State', 'PIN Code', 'Dispatch After date', 'Dispatch by date', 'Form requirement', 'Tracking ID', 'Package Length (cm)', 'Package Breadth (cm)', 'Package Height (cm)', 'Package Weight (kg)', 'Ready to Make', 'With Attachment', 'Master SKU']


In [13]:
def update_inventory(df_sales, df_inventory):
    if "Master SKU" not in df_sales.columns:
        print("Error: 'Master SKU' column not found in sales data")
        return df_inventory

    if "msku" not in df_inventory.columns:  # Ensure inventory has 'msku'
        print("Error: 'msku' column not found in inventory data")
        return df_inventory

    for index, row in df_sales.iterrows():
        msku = row["Master SKU"]
        quantity_sold = row["Quantity"]

        if msku in df_inventory["msku"].values:
            df_inventory.loc[df_inventory["msku"] == msku, "Opening Stock"] -= quantity_sold

    return df_inventory

# Ensure column names are stripped of spaces
df_sales_amazon.columns = df_sales_amazon.columns.str.strip()
df_sales_flipkart.columns = df_sales_flipkart.columns.str.strip()
df_sales_meesho.columns = df_sales_meesho.columns.str.strip()

# Apply the update function
df_inventory = update_inventory(df_sales_amazon, df_inventory)
df_inventory = update_inventory(df_sales_flipkart, df_inventory)
df_inventory = update_inventory(df_sales_meesho, df_inventory)


C:\Users\Tejas\AppData\Local\Temp\ipykernel_23700\3736697072.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inventory.loc[df_inventory["msku"] == msku, "Opening Stock"] -= quantity_sold


In [39]:
def process_combos(df_sales, df_combos, df_inventory):
    for index, row in df_sales.iterrows():
        msku = row["msku"]  # Change "Master SKU" to "msku"
        quantity_sold = row["quantity"]

        # Check if the SKU is part of a combo
        if msku in df_combos["combo"].values:  
            combo_row = df_combos[df_combos["combo"] == msku].iloc[0]

            # Iterate over SKU columns (sku1, sku2, ..., sku14)
            for i in range(1, 15):  
                component_sku = combo_row.get(f"sku{i}")
                if pd.notna(component_sku):  
                    df_inventory.loc[df_inventory["msku"] == component_sku, "Current Stock"] -= quantity_sold  

    return df_inventory


In [40]:
print(df_combos.columns.tolist())


['Combo', 'SKU1', 'SKU2', 'SKU3', 'SKU4', 'SKU5', 'SKU6', 'SKU7', 'SKU8', 'SKU9', 'SKU10', 'SKU11', 'SKU12', 'SKU13', 'SKU14', 'Status', 'Unnamed: 16']


In [45]:
import pandas as pd

def process_combos(df_sales, df_combos, df_inventory):
    # Ensure 'Current Stock' column exists
    if "Current Stock" not in df_inventory.columns:
        df_inventory["Current Stock"] = df_inventory["Opening Stock"].copy()
    
    for index, row in df_sales.iterrows():
        msku = row.get("Master SKU")  # Use .get() to avoid KeyError
        quantity_sold = row.get("Quantity", 0)

        if pd.isna(msku) or pd.isna(quantity_sold):  # Skip invalid rows
            continue

        if msku in df_combos["Combo"].values:
            combo_components = df_combos[df_combos["Combo"] == msku]

            for _, component in combo_components.iterrows():
                for i in range(1, 15):  # Loop through SKU1 to SKU14
                    component_sku = component.get(f"SKU{i}")  # Get SKU dynamically
                    if pd.notna(component_sku):  # Ignore NaN values
                        # Ensure the SKU exists in inventory before subtracting
                        mask = df_inventory["msku"] == component_sku
                        if mask.any():
                            df_inventory.loc[mask, "Current Stock"] -= quantity_sold

    return df_inventory

# Process sales data for different platforms
df_inventory = process_combos(df_sales_amazon, df_combos, df_inventory)
df_inventory = process_combos(df_sales_flipkart, df_combos, df_inventory)
df_inventory = process_combos(df_sales_meesho, df_combos, df_inventory)


C:\Users\Tejas\AppData\Local\Temp\ipykernel_23700\1082992265.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inventory["Current Stock"] = df_inventory["Opening Stock"].copy()


In [44]:
print(df_inventory.columns)


Index(['Product Name', 'msku', 'Opening Stock', 'Buffer Stock', 'TLCQ', 'BLR7',
       'BLR8', 'BOM5', 'BOM7', 'CCU1', 'CCX1', 'DEL4', 'DEL5', 'DEX3', 'PNQ2',
       'PNQ3', 'SDED', 'SDEE', 'XHJ9'],
      dtype='object', name=0)


In [52]:
df_returns.columns = df_returns.columns.str.strip()
df_inwards.columns = df_inwards.columns.str.strip()
df_inventory.columns = df_inventory.columns.str.strip()

# Debugging: Print column names
print("df_returns columns:", df_returns.columns.tolist())
print("df_inwards columns:", df_inwards.columns.tolist())
print("df_inventory columns:", df_inventory.columns.tolist())

# Rename columns to match expected names
column_mappings = {
    "Msku": "Master SKU",  # Some files use 'Msku', while others use 'msku'
    "msku": "Master SKU",  # Ensure consistency with 'msku'
    "QTY": "Returned Quantity",  # Rename quantity column if needed
    "NewStock": "New Stock",  # Rename variations
    "CurrentStock": "Current Stock"
}

df_returns.rename(columns=column_mappings, inplace=True)
df_inwards.rename(columns=column_mappings, inplace=True)
df_inventory.rename(columns=column_mappings, inplace=True)

# Validate column presence
required_columns = {
    "df_returns": {"Master SKU", "Returned Quantity"},
    "df_inwards": {"Master SKU", "New Stock"},
    "df_inventory": {"Master SKU", "Current Stock"},
}

for df_name, cols in required_columns.items():
    missing_cols = cols - set(eval(df_name).columns)
    if missing_cols:
        raise KeyError(f"Missing columns in {df_name}: {missing_cols}")

# Convert 'Master SKU' to string to avoid merge issues
df_returns["Master SKU"] = df_returns["Master SKU"].astype(str)
df_inwards["Master SKU"] = df_inwards["Master SKU"].astype(str)
df_inventory["Master SKU"] = df_inventory["Master SKU"].astype(str)

# Merge returns and inwards stock data
df_inventory = df_inventory.merge(df_returns[["Master SKU", "Returned Quantity"]], on="Master SKU", how="left")
df_inventory = df_inventory.merge(df_inwards[["Master SKU", "New Stock"]], on="Master SKU", how="left")

# Fill NaN values with 0
df_inventory["Returned Quantity"] = df_inventory["Returned Quantity"].fillna(0)
df_inventory["New Stock"] = df_inventory["New Stock"].fillna(0)

# Ensure numeric columns for calculations
df_inventory["Returned Quantity"] = df_inventory["Returned Quantity"].astype(float)
df_inventory["New Stock"] = df_inventory["New Stock"].astype(float)
df_inventory["Current Stock"] = df_inventory["Current Stock"].astype(float)

# Calculate Final Stock
df_inventory["Final Stock"] = df_inventory["Current Stock"] + df_inventory["Returned Quantity"] + df_inventory["New Stock"]

# Debugging Output
print(df_inventory.head())

df_returns columns: ['Master SKU', 'Returned Quantity', 'Warehouse', 'Condition', 'Type']
df_inwards columns: ['ITEM NAME', 'msku', 'New Stock', 'Purpose', 'Date']
df_inventory columns: ['Product Name', 'msku', 'Opening Stock', 'Buffer Stock', 'TLCQ', 'BLR7', 'BLR8', 'BOM5', 'BOM7', 'CCU1', 'CCX1', 'DEL4', 'DEL5', 'DEX3', 'PNQ2', 'PNQ3', 'SDED', 'SDEE', 'XHJ9', 'Current Stock']
     Product Name                      Master SKU Opening Stock Buffer Stock  \
0  Chimmy Pillow   CSTE_0001_ST_Bts_Pillow_Chimmy           193          NaN   
1    Tata Pillow     CSTE_0002_ST_Bts_Pillow_Tata             0          NaN   
2    Koya Pillow     CSTE_0003_ST_Bts_Pillow_Koya             0          NaN   
3    Mang Pillow     CSTE_0004_ST_Bts_Pillow_Mang            21          NaN   
4  Shooky Pillow   CSTE_0005_ST_Bts_Pillow_Shooky             0          NaN   

  TLCQ BLR7 BLR8 BOM5 BOM7 CCU1  ... DEX3 PNQ2 PNQ3 SDED SDEE XHJ9  \
0  192    0    0    0   -1    0  ...    0    0    0    0    0    0  

In [54]:
print("", df_outflow.columns.tolist())
print("", df_inventory.columns.tolist())


 ['Msku', 'QTY', 'Purpose', 'Date', 'Warehouse Name(IF FBA)']
 ['Product Name', 'Master SKU', 'Opening Stock', 'Buffer Stock', 'TLCQ', 'BLR7', 'BLR8', 'BOM5', 'BOM7', 'CCU1', 'CCX1', 'DEL4', 'DEL5', 'DEX3', 'PNQ2', 'PNQ3', 'SDED', 'SDEE', 'XHJ9', 'Current Stock', 'Returned Quantity', 'New Stock', 'Final Stock']


In [55]:
import pandas as pd

# Standardize column names (strip spaces)
df_outflow.columns = df_outflow.columns.str.strip()
df_inventory.columns = df_inventory.columns.str.strip()

# Debugging: Print column names before merging
print("df_outflow columns:", df_outflow.columns.tolist())
print("df_inventory columns:", df_inventory.columns.tolist())

# Rename columns to match expected names
df_outflow.rename(columns={"Msku": "Master SKU", "QTY": "Outflow Quantity"}, inplace=True)

# Validate column presence
required_columns = {
    "df_outflow": {"Master SKU", "Outflow Quantity"},
    "df_inventory": {"Master SKU", "Final Stock"},
}

for df_name, cols in required_columns.items():
    missing_cols = cols - set(eval(df_name).columns)
    if missing_cols:
        raise KeyError(f"Missing columns in {df_name}: {missing_cols}")

# Convert 'Master SKU' to string for consistency
df_outflow["Master SKU"] = df_outflow["Master SKU"].astype(str)
df_inventory["Master SKU"] = df_inventory["Master SKU"].astype(str)

# Merge outflow stock data
df_inventory = df_inventory.merge(df_outflow[["Master SKU", "Outflow Quantity"]], on="Master SKU", how="left")

# Fill NaN values with 0
df_inventory["Outflow Quantity"] = df_inventory["Outflow Quantity"].fillna(0).astype(float)

# Subtract Outflow from Final Stock
df_inventory["Final Stock"] -= df_inventory["Outflow Quantity"]

# Debugging Output
print(df_inventory.head())


df_outflow columns: ['Msku', 'QTY', 'Purpose', 'Date', 'Warehouse Name(IF FBA)']
df_inventory columns: ['Product Name', 'Master SKU', 'Opening Stock', 'Buffer Stock', 'TLCQ', 'BLR7', 'BLR8', 'BOM5', 'BOM7', 'CCU1', 'CCX1', 'DEL4', 'DEL5', 'DEX3', 'PNQ2', 'PNQ3', 'SDED', 'SDEE', 'XHJ9', 'Current Stock', 'Returned Quantity', 'New Stock', 'Final Stock']
     Product Name                      Master SKU Opening Stock Buffer Stock  \
0  Chimmy Pillow   CSTE_0001_ST_Bts_Pillow_Chimmy           193          NaN   
1    Tata Pillow     CSTE_0002_ST_Bts_Pillow_Tata             0          NaN   
2    Koya Pillow     CSTE_0003_ST_Bts_Pillow_Koya             0          NaN   
3    Mang Pillow     CSTE_0004_ST_Bts_Pillow_Mang            21          NaN   
4  Shooky Pillow   CSTE_0005_ST_Bts_Pillow_Shooky             0          NaN   

  TLCQ BLR7 BLR8 BOM5 BOM7 CCU1  ... PNQ2 PNQ3 SDED SDEE XHJ9 Current Stock  \
0  192    0    0    0   -1    0  ...    0    0    0    0    0         193.0   
1    0  

In [56]:
df_inventory.to_excel("processed_inventory.xlsx", index=False)
print("Final inventory saved successfully!")


Final inventory saved successfully!
